In [1]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merging import add
from keras.models import Model, load_model
from keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm_notebook as tqdm #to check loop progress
tqdm().pandas()

c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Load the document file into memory
def load_fp(filename):
   # Open file to read
   file = open(filename, 'r')
   text = file.read()
   file.close()
   return text

# get all images with their captions
def img_capt(filename):
   file = load_fp(filename)
   captions = file.split('\n')
   descriptions ={}
   for caption in captions[:-1]:
      img, caption = caption.split('\t')
      if img[:-2] not in descriptions:
         descriptions[img[:-2]] = [ caption ]
      else:
         descriptions[img[:-2]].append(caption)
   
   return descriptions

#Data cleaning function will convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
def txt_clean(captions):
   table = str.maketrans('','',string.punctuation)
   for img,caps in captions.items():
      for i,img_caption in enumerate(caps):
         img_caption.replace("-"," ")
         descp = img_caption.split()
         #uppercase to lowercase
         descp = [wrd.lower() for wrd in descp]
         #remove punctuation from each token
         descp = [wrd.translate(table) for wrd in descp]
         #remove hanging 's and a
         descp = [wrd for wrd in descp if(len(wrd)>1)]
         #remove words containing numbers with them
         descp = [wrd for wrd in descp if(wrd.isalpha())]
         #converting back to string
         img_caption = ' '.join(descp)
         captions[img][i]= img_caption
   return captions

def txt_vocab(descriptions):
   # To build vocab of all unique words
   vocab = set()
   for key in descriptions.keys():
      [vocab.update(d.split()) for d in descriptions[key]]
   return vocab

#To save all descriptions in one file
def save_descriptions(descriptions, filename):
   lines = list()
   for key, desc_list in descriptions.items():
      for desc in desc_list:
         lines.append(key + '\t' + desc )
   data = "\n".join(lines)
   file = open(filename,"w")
   file.write(data)
   file.close()

# Set these path according to project folder in you system, like i create a folder with my name shikha inside D-drive
dataset_text = "C:\\Users\\asus\\Downloads\\Flickr8k_text"
dataset_images = "C:\\Users\\asus\\Downloads\\Flickr8k_Dataset\\Flicker8k_Dataset"
#to prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#map them into descriptions dictionary 
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
#to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")


Length of descriptions = 8092
Length of vocabulary =  8763


In [11]:
def extract_features(dirc):
    model = Xception( include_top=False, pooling='avg' )
    features = {}
    for pic in tqdm(os.listdir(dirc)):
        file = dirc + "/" + pic
        image = Image.open(file)
        image = image.resize((299,299))
        image = np.expand_dims(image, axis=0)
        #image = preprocess_input(image)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        features[pic] = feature
    return features

#2048 feature vector
features = extract_features(dataset_images)
dump(features, open("features.p","wb"))

#to directly load the features from the pickle file.
features = load(open("features.p","rb"))

c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


1/1 [==============================] - 1s 798ms/step



In [23]:
#load the data
def load_photos(filename):
    file = load_fp(filename)
    photos = file.split("\n")[:-1]
    return photos

def load_clean_descriptions(filename, photos):
    #loading clean_descriptions
    file = load_fp(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
                desc = ' ' + " ".join(image_caption) + ' '
                descriptions[image].append(desc)

    return descriptions

def load_features(photos):
    #loading all features
    all_features = load(open("features.p","rb"))

    #selecting only needed features
    features = {k:all_features[k] for k in photos}

    return features

filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [24]:
#convert dictionary to clear list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

#creating tokenizer class
#this will vectorise text corpus
#each integer will represent token in dictionary
from keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size) #The size of our vocabulary is 7577 words.
#calculate maximum length of descriptions to decide the model structure parameters.
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_lengths = max_length(descriptions)
print(max_lengths) #Max_length of description is 32
print(descriptions.items)

3846
32
<built-in method items of dict object at 0x0000015263E2CE08>


In [29]:
#data generator, used by model.fit_generator()

def data_generator(descriptions, features, tokenizer, max_length):
      while True :
            for key, description_list in descriptions.items():
                  #retrieve photo features
                  print(key, description_list)
                  feature = features[key][0]
                  inp_image, inp_seq, op_word = create_sequences(tokenizer, max_length, description_list, feature)
                  yield [[inp_image, inp_seq], op_word]

def create_sequences(tokenizer, max_length, desc_list, feature):
      x_1, x_2, y = list(), list(), list()
      # move through each description for the image
      for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
      # divide one sequence into various X,y pairs
      for i in range(1, len(seq)):
            # divide into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            x_1.append(feature)
            x_2.append(in_seq)
            y.append(out_seq)

      return np.array(x_1), np.array(x_2), np.array(y)

#To check the shape of the input and output for your model
[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_lengths))
print(a.shape, b.shape, c.shape)
#((47, 2048), (47, 32), (47, 7577))

1000268201_693b08cb0e.jpg [' child in pink dress is climbing up set of stairs in an entry way ']
(13, 2048) (13, 32) (13, 3846)


In [30]:
from keras.utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):
  # features from the CNN model compressed from 2048 to 256 nodes
  inputs1 = Input(shape=(2048,))
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256, activation='relu')(fe1)
  # LSTM sequence model
  inputs2 = Input(shape=(max_length,))
  se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)
  # Merging both models
  decoder1 = add([fe2, se3])
  decoder2 = Dense(256, activation='relu')(decoder1)
  outputs = Dense(vocab_size, activation='softmax')(decoder2)
  # merge it [image, seq] [word]
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  # summarize model
  print(model.summary())
  plot_model(model, to_file='model.png', show_shapes=True)
  
  return model

In [33]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_lengths)
model = define_model(vocab_size, max_lengths)
epochs = 10
steps = len(train_descriptions)
# creating a directory named models to save our models
#os.mkdir("models_simple")
for i in range(epochs):
   generator = data_generator(train_descriptions, train_features, tokenizer, max_lengths)
   model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
   model.save("models_simple/model_" + str(i) + ".h5")

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 3846
Description Length:  32
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32)]         0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 32, 256)      984576      ['input_10[0][0]']               
                                                                                                  
 dropout_3 (Dropout)            (None, 2048)         0           ['input_9[0][0]']  

c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


1001773457_577c3a7d70.jpg [' black dog and spotted dog are fighting ']
3163/6000 [==============>...............] - ETA: 18:54 - loss: 5.71163154709407_9b0778cbeb.jpg [' bald man dribbles basketball while wearing miami jersey ']


InvalidArgumentError: Graph execution error:

TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1077, in generator_py_func
    f"`generator` yielded an element of shape {ret_array.shape} "

TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_1621200]